## 8. Model Optimization with Optuna

In [ ]:
def objective(trial):
    # Hyperparameters to optimize
    hidden_dim = trial.suggest_int('hidden_dim', 32, 256)
    num_heads = trial.suggest_int('num_heads', 2, 8)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.7)
    edge_threshold = trial.suggest_float('edge_threshold', 0.3, 0.8)
    
    # Create model with trial parameters
    model = GNNModel(
        input_dim=X.shape[1],
        hidden_dim=hidden_dim,
        num_heads=num_heads
    ).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    val_losses = []
    for epoch in range(100):
        train_loss = train_model(model, train_loader, optimizer, device)
        val_loss = evaluate_model(model, val_loader, device)
        val_losses.append(val_loss)
        
        # Early stopping
        if epoch > 10 and val_losses[-1] > val_losses[-2]:
            break
    
    return min(val_losses)

## 9. Advanced Feature Engineering

In [ ]:
def engineer_advanced_features(df):
    """
    Create advanced features for the model
    """
    # Age-related interactions
    df['age_comorbidity_interaction'] = df['admit_age'] * df['age_adj_comorbidity_score']
    
    # Injury severity composite
    df['injury_severity_score'] = (
        df['brain_injury_mild/_moderate'].astype(float) * 1 +
        df['brain_injury_severe'].astype(float) * 2
    )
    
    # Medical complexity index
    complexity_columns = [
        'diabetes', 'hypertension', 'heart_disease',
        'neurological_disorder', 'psychiatric_disorder'
    ]
    df['medical_complexity_index'] = df[complexity_columns].sum(axis=1)
    
    # Functional status composite
    df['functional_status_score'] = (
        df['motor_score'].astype(float) +
        df['pre-injury_function'].astype(float)
    )
    
    return df

# Apply advanced feature engineering
df = engineer_advanced_features(df)

## 10. Custom Loss Function

In [ ]:
class WeightedMSELoss(torch.nn.Module):
    def __init__(self, threshold=7):
        super(WeightedMSELoss, self).__init__()
        self.threshold = threshold
        
    def forward(self, pred, target):
        # Higher weight for longer stays
        weights = torch.where(target > self.threshold,
                            torch.ones_like(target) * 2.0,
                            torch.ones_like(target))
        
        return torch.mean(weights * (pred - target) ** 2)

# Use custom loss function
criterion = WeightedMSELoss()

## 11. Patient Subgroup Analysis

In [ ]:
def analyze_patient_subgroups(embeddings, labels, n_clusters=5):
    """
    Perform patient subgroup analysis using clustering
    """
    from sklearn.cluster import KMeans
    
    # Cluster patients
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(embeddings)
    
    # Analyze characteristics of each cluster
    cluster_stats = []
    for i in range(n_clusters):
        cluster_mask = clusters == i
        stats = {
            'cluster': i,
            'size': np.sum(cluster_mask),
            'avg_los': np.mean(labels[cluster_mask]),
            'std_los': np.std(labels[cluster_mask])
        }
        cluster_stats.append(stats)
    
    return pd.DataFrame(cluster_stats)

## 12. Real-time Prediction Setup

In [ ]:
class RealTimePredictionPipeline:
    def __init__(self, model, scaler, feature_names):
        self.model = model
        self.scaler = scaler
        self.feature_names = feature_names
    
    def preprocess_patient(self, patient_data):
        """
        Preprocess a single patient's data
        """
        # Ensure all features are present
        for feature in self.feature_names:
            if feature not in patient_data:
                raise ValueError(f"Missing feature: {feature}")
        
        # Convert to array and scale
        X = np.array([patient_data[f] for f in self.feature_names]).reshape(1, -1)
        X_scaled = self.scaler.transform(X)
        return torch.FloatTensor(X_scaled)
    
    def predict(self, patient_data):
        """
        Make real-time prediction for a patient
        """
        self.model.eval()
        with torch.no_grad():
            X = self.preprocess_patient(patient_data)
            pred = self.model(X)
            return pred.item()

## 13. Model Evaluation and Validation

In [ ]:
def evaluate_comprehensive(model, test_loader, device):
    """
    Comprehensive model evaluation
    """
    model.eval()
    predictions = []
    actuals = []
    
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.edge_index, data.batch)
            predictions.extend(out.cpu().numpy())
            actuals.extend(data.y.cpu().numpy())
    
    predictions = np.array(predictions)
    actuals = np.array(actuals)
    
    # Calculate metrics
    mse = np.mean((predictions - actuals) ** 2)
    mae = np.mean(np.abs(predictions - actuals))
    r2 = 1 - np.sum((actuals - predictions) ** 2) / np.sum((actuals - np.mean(actuals)) ** 2)
    
    return {
        'mse': mse,
        'rmse': np.sqrt(mse),
        'mae': mae,
        'r2': r2
    }